In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd 
import numpy as np 
import time, random, re, csv
import statsmodels.api as sm
import seaborn as sns 
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.model_selection import (cross_val_score, train_test_split, KFold, GridSearchCV)
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt 
import scipy.stats as stats

In [4]:
DASS = pd.read_csv("/Users/che-yuliu/Downloads/Metis Business Project_Che-Yu Liu - DASS Raw Data.csv")

In [5]:
DASS.head(10)

,participant_id,Q1A,Q2A,Q3A,Q4A,Q5A,Q6A,Q7A,Q8A,Q9A,...,screensize,uniquenetworklocation,hand,religion,orientation,race,voted,married,familysize,major
0,1,4,4,2,4,4,4,4,4,2,...,1,1,1,12,1,10,2,1,2,NaN
1,2,4,1,2,3,4,4,3,4,3,...,2,1,2,7,0,70,2,1,4,NaN
2,3,3,1,4,1,4,3,1,3,2,...,2,1,1,4,3,60,1,1,3,NaN
3,4,2,3,2,1,3,3,4,2,3,...,2,1,2,4,5,70,2,1,5,biology
4,5,2,2,3,4,4,2,4,4,4,...,2,2,3,10,1,10,2,1,4,Psychology
5,6,1,1,2,1,3,1,1,3,3,...,2,1,1,4,1,70,2,1,4,NaN
6,7,1,1,2,3,4,1,3,3,3,...,2,1,1,7,2,60,2,1,4,Mechatronics engeenerieng
7,8,1,1,1,1,3,2,2,1,1,...,2,1,1,2,2,60,1,1,2,Music
8,9,4,4,3,4,3,4,4,4,4,...,1,1,1,12,2,70,2,1,4,Psychology
9,10,3,2,4,1,4,4,3,4,4,...,2,1,1,2,2,60,2,1,3,computer programming


## Data Cleaning

Replace incorrect replies based on EDA conducted on the each of these aspects. 
(Replaced based on similar distribution of scores)

In [7]:
DASS["gender"] = DASS["gender"].replace(0, 3)

In [8]:
DASS["education"] = DASS["education"].replace(0, 3)

In [9]:
DASS["married"] = DASS["gender"].replace(0, 1)

In [23]:
DASS["engnat"] = DASS["engnat"].replace(0,2)

## Creating Baseline Model

In [24]:
X, X_test, y, y_test = train_test_split(DASS[["gender","education","married","race","orientation","engnat"]], DASS["DASS Total Score"], 
                                                    test_size=0.2, 
                                                    random_state=10)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.25, random_state = 3)

In [25]:
X_train = sm.add_constant(X_train)

In [26]:
# Used statsmodels.api to get basic summary info about the regression
train_model1 = sm.OLS(y_train, X_train)
train_model1_fit = train_model1.fit()
print(train_model1_fit.summary())

                            OLS Regression Results                            
Dep. Variable:       DASS Total Score   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.054
Method:                 Least Squares   F-statistic:                     275.8
Date:                Tue, 06 Sep 2022   Prob (F-statistic):          1.08e-287
Time:                        13:33:59   Log-Likelihood:            -1.1442e+05
No. Observations:               23865   AIC:                         2.289e+05
Df Residuals:                   23859   BIC:                         2.289e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          99.2709      1.311     75.711      

In [27]:
LM1 = LinearRegression()
LM1.fit(X_train,y_train)
print("R2 Score:", LM1.score(X_train,y_train))


R2 Score: 0.054630981084995534


In [28]:
LM1.fit(X_val,y_val)
print("R2 Score:", LM1.score(X_val,y_val))

R2 Score: 0.04741238003898107


In [29]:
LM1.fit(X_test,y_test)
print("R2 Score:", LM1.score(X_test,y_test))

R2 Score: 0.044977662053295786


In [32]:
list(zip(X_train[["gender","education","married","race","orientation","engnat"]],LM1.coef_))

[('gender', 2.9770469964287507),
 ('education', -5.926127418825868),
 ('married', 2.9770469964287614),
 ('race', -0.009110284899415788),
 ('orientation', 1.8697509113127289),
 ('engnat', 0.3745321862886634)]